# Quantify MTM1M3 outer loop times.  Times should be less than 20 ms.
Craig Lage 15-Jan-24

In [ ]:
import sys, time, os, asyncio, copy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient
from lsst.summit.utils import dayObsIntToString
from lsst.summit.utils.tmaUtils import TMAEventMaker
from lsst.summit.utils.efdUtils import calcNextDay 

# The cell below runs the whole night from the start of the first event to the end of the last event.

In [ ]:
client = EfdClient("usdf_efd")
eventMaker = TMAEventMaker()

startDay = 20231116
endDay = 20231117

pdf = PdfPages("/home/c/cslage/u/MTM1M3/data/OuterLoopTimes_15Jan24.pdf")
fig = plt.figure(figsize = (5,5))

dayObs = startDay
while dayObs < endDay:
    events = eventMaker.getEvents(dayObs)
    if len(events) < 100:
        dayObs = calcNextDay(dayObs)
        continue
    start = events[0].begin
    end = events[-1].end
    outerLoopTime = await client.select_time_series("lsst.sal.MTM1M3.outerLoopData", \
                                                ['executionTime'], \
                                             start, end)
    times = outerLoopTime['executionTime'].values * 1000.0
    times = times[times < 1000.0]
    longTimes = times[times>20.0]
    ax = fig.subplots(1,1)
    ax.set_title(f"Outer Loop Times - {dayObs}")
    ax.hist(times, bins = 1000)
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_xlabel("Outer Loop Time (ms)")
    ax.set_xlim(1,100)
    ax.set_ylim(1,1E7)
    ax.plot([20, 20],[0,1E6], ls='--', color='black')
    ax.text(2, 1E6, f"{(len(longTimes) / len(times) * 100.0):.2f} % of data are \n> 20ms")
    pdf.savefig(fig)  # saves the current figure into a pdf page
    print(f"Plot for dayObs {dayObs} finished!")
    plt.clf()
    dayObs = calcNextDay(dayObs)
pdf.close()

    

# The cell below only looks at the times during events.

In [ ]:
client = EfdClient("usdf_efd")
eventMaker = TMAEventMaker()

startDay = 20231116
endDay = 20240109

pdf = PdfPages("/home/c/cslage/u/MTM1M3/data/OuterLoopTimes_19Jan24.pdf")
fig = plt.figure(figsize = (5,5))

dayObs = startDay
while dayObs < endDay:
    events = eventMaker.getEvents(dayObs)
    if len(events) < 100:
        dayObs = calcNextDay(dayObs)
        continue
    times = np.array([])
    for i, event in enumerate(events):
        try:
            start = event.begin
            end = event.end
            outerLoopTime = await client.select_time_series("lsst.sal.MTM1M3.outerLoopData", \
                                                        ['executionTime'], \
                                                     start, end)
            theseTimes = outerLoopTime['executionTime'].values * 1000.0
            if len(times) == 0:
                times = theseTimes
            else:
                times = np.concatenate([times, theseTimes])
        except:
            continue
    times = times[times < 1000.0]
    longTimes = times[times>20.0]
    ax = fig.subplots(1,1)
    ax.set_title(f"Outer Loop Times - {dayObs}")
    ax.hist(times, bins = 1000)
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_xlabel("Outer Loop Time (ms)")
    ax.set_xlim(1,100)
    ax.set_ylim(1,1E7)
    ax.plot([20, 20],[0,1E6], ls='--', color='black')
    ax.text(2, 1E6, f"{(len(longTimes) / len(times) * 100.0):.2f} % of data are \n> 20ms")
    pdf.savefig(fig)  # saves the current figure into a pdf page
    print(f"Plot for dayObs {dayObs} finished!")
    plt.clf()
    dayObs = calcNextDay(dayObs)
pdf.close()

    

# The cell below looks at the MTM1M3 states.

In [ ]:
%matplotlib inline
dayObs = 20240104
start = Time(f"{dayObsIntToString(dayObs)} 19:00:00Z", scale='utc')
end = Time(f"{dayObsIntToString(calcNextDay(dayObs))} 10:00:00Z", scale='utc')
state = await client.select_time_series('lsst.sal.MTM1M3.logevent_detailedState', ['*'], start, end)
fig, ax = plt.subplots(1,1,figsize=(12,5))
ax.set_title(f"MTM1M3 states and events {dayObs} \nRed dots are state changes dotted black lines are TMAEvents")
state['detailedState'].plot(ax = ax, marker='o', color='r', ls='')
states = state['detailedState'].values
for i in range(len(states) - 1):
    ax.hlines(states[i], state.index[i], state.index[i+1], color='r')

events = eventMaker.getEvents(dayObs)
for i,event in enumerate(events):
    if Time(event.begin).unix_tai < start.unix_tai:
        continue
    if i % 20 == 0:
        ax.axvline(event.begin.isot, ls = '--', color='black')


xml_states = ['DisabledState','FaultState','OfflineState','StandbyState','ParkedState','RaisingState', \
 'ActiveState','LoweringState','ParkedEngineeringState','RaisingEngineeringState', \
 'ActiveEngineeringState','LoweringEngineeringState','LoweringFaultState', \
 'ProfileHardpointCorrectionState','PausedRaisingState','PausedRaisingEngineeringState', \
 'PausedLoweringState','PausedLoweringEngineeringState', '']
ax2 = ax.twinx()
ax.set_yticks(list(range(len(xml_states))))
ax2.set_yticks(list(range(len(xml_states))))
ax2.set_yticklabels(xml_states)
ax.set_ylabel("MTM1M3 State")
plt.subplots_adjust(right = 0.6)
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/MTM1M3_States_{dayObs}.png")